In [9]:
import pandas as pd
raw_data_events = pd.read_csv("./data/events.csv")

In [12]:
raw_data_events.columns

Index(['id', 'user_id', 'sequence_number', 'session_id', 'created_at',
       'ip_address', 'city', 'state', 'postal_code', 'browser',
       'traffic_source', 'uri', 'event_type'],
      dtype='object')

In [13]:
raw_data_events.head()

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,1654067,NaN,3,9b5dcc78-e4f7-47df-8e3e-80b54de7f5c3,2022-03-03 08:24:00.000000 UTC,81.166.35.255,São Paulo,São Paulo,02675-031,Firefox,Facebook,/cancel,cancel
1,1634620,NaN,3,9280f069-144d-44cf-8d83-81910cd114fc,2023-10-24 17:07:00.000000 UTC,141.167.176.122,São Paulo,São Paulo,02675-031,Safari,Facebook,/cancel,cancel
2,2419377,NaN,3,b32517a3-086b-4ca2-8a69-4665fd7c8cfe,2023-04-03 14:44:00.000000 UTC,207.12.88.71,São Paulo,São Paulo,02675-031,Safari,Email,/cancel,cancel
3,1643719,NaN,3,91e7c791-0ad0-4dba-ac21-ab28bc1c288d,2022-05-04 12:31:00.000000 UTC,87.43.237.190,São Paulo,São Paulo,02675-031,Firefox,Email,/cancel,cancel
4,2165272,NaN,3,6f3f52fd-a887-43c0-963f-5280d625f38f,2023-01-07 18:30:00.000000 UTC,153.110.117.58,São Paulo,São Paulo,02675-031,Safari,Facebook,/cancel,cancel


In [16]:
raw_data_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47948 entries, 0 to 47947
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               47948 non-null  int64  
 1   user_id          25115 non-null  float64
 2   sequence_number  47948 non-null  int64  
 3   session_id       47948 non-null  object 
 4   created_at       47948 non-null  object 
 5   ip_address       47948 non-null  object 
 6   city             47487 non-null  object 
 7   state            47948 non-null  object 
 8   postal_code      47948 non-null  object 
 9   browser          47948 non-null  object 
 10  traffic_source   47948 non-null  object 
 11  uri              47948 non-null  object 
 12  event_type       47948 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 4.8+ MB


(47948, 13)

Except for userId, most columns are filled.

## Grouping by event type

In [62]:
by_event_type = raw_data_events.groupby(by=["state", "event_type"])["id"].count().sort_values(ascending=False)
by_event_type.to_csv(path_or_buf="./export/event_count_by_state.csv")

## Concentration of sales

In [61]:
only_sales = raw_data_events[raw_data_events["event_type"] == "cart"]
sales_count_by_source = only_sales.groupby(by=["state", "traffic_source"])["id"].count().sort_values(ascending=False)
sales_count_by_source.to_csv(path_or_buf="./export/sales_count_by_source.csv")